In [1]:
import sys
import folium
from folium import plugins
from folium.plugins import FloatImage
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime
import pandas as pd
import numpy
import geopandas as gp
from geopy.geocoders import Nominatim
from datetime import datetime
import time
import re
import smtplib

now = datetime.now()
time_date_now = now.strftime("%d/%m/%Y %H:%M:%S")

In [2]:
def Geocoder(dataset, dataset_name):
    print(f"Beginning Geocoding of {dataset_name}...")

    for i in range(0,len(dataset)):
        if i == round(len(dataset)*0.25):
            print("25% Complete")
        if i == round(len(dataset)*0.50):
            print("50% Complete")
        if i == round(len(dataset)*0.75):
            print("75% Complete")
        if i == round(len(dataset)*1.00):
            print("100% Complete")
        else:
            pass

        is_null = pd.isnull([dataset.at[i, 'Latitude']])
        if is_null == True:
            try:
                    time.sleep(0.1)
                    locator = Nominatim(user_agent="myGeocoder")
                    location = locator.geocode([dataset.at[i, 'Postcode']])
                    dataset.at[i, "Latitude"]= location.latitude
                    dataset.at[i, "Longitude"]= location.longitude
                    #print (f" Postcode: {dataset.at[i, 'Postcode']} Latitude = {location.latitude}, Longitude = {location.longitude}")
                    #print (f" {i} of {len(dataset)} | Postcode: {dataset.at[i, 'Postcode']} Latitude = {location.latitude}, Longitude = {location.longitude}")
            except:
                    #print(f" Postcode: {dataset.at[i, 'Postcode']} **No Value Found**")
                    dataset.at[i, "Latitude"]= 53.827712
                    dataset.at[i, "Longitude"]= 1.997828
                    #print(f" Postcode: {dataset.at[i, 'Postcode']} **Position plotted in the middle of the North Sea, please revisit this and correct the Postcode!**")
        else:
          #print("No new Postcode detected, moving on...")
          pass


    print(f"Geocoding {dataset_name} Complete...")

In [3]:
# Colours

wasteonline_blue = "#4A6BFF"
howarth_green = "#2EA739"

#{'cadetblue', 'lightred', 'orange', 'pink', 'darkpurple', 'beige', 'lightgreen', 'darkblue', 'darkred', 'blue', 
# 'darkgreen', 'gray', 'lightgray', 'lightblue', 'red', 'purple', 'black', 'white', 'green'}.


In [4]:
# Create a map
main_map = folium.Map(location=[55.087240, -3.180290], tiles='OpenStreetMap', zoom_start=6)
folium.TileLayer(location=[55.087240, -3.180290], tiles='https://api.mapbox.com/styles/v1/niallxd/ck9izkw7905141imf0qf6boj8/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoibmlhbGx4ZCIsImEiOiJjazlpemZkbnEwMDBzM2VudXdzaWdhZWozIn0.wd2EYP1KL4enBLfLvFz3Ag'
               , attr= "Map Style By NiallxD - Made with Mapbox", name="Dark Mode", detect_retina=False, overlay=True, Show=True).add_to(main_map)
folium.TileLayer(location=[55.087240, -3.180290], tiles='https://api.mapbox.com/styles/v1/niallxd/ck9j00a9z0aq71io9mye045sr/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoibmlhbGx4ZCIsImEiOiJjazlpemZkbnEwMDBzM2VudXdzaWdhZWozIn0.wd2EYP1KL4enBLfLvFz3Ag'
               , attr= "Map Style By NiallxD - Made with Mapbox", name="Light Mode", detect_retina=False, overlay=True, Show=True).add_to(main_map)

supplierlayer = folium.FeatureGroup("WasteOnline Suppliers").add_to(main_map)
howarthlayer = folium.FeatureGroup("Howarth Timber Branches").add_to(main_map)
howarthradlayer = folium.FeatureGroup("Howarth Timber - 15 Mile Radius", show=False).add_to(main_map)

measure_tool = folium.plugins.MeasureControl(position='topleft').add_to(main_map)

folium.LayerControl(collapsed=False).add_to(main_map)


In [5]:
# Add in a legend image

legend_png = 'https://i.imgur.com/MqIVnoD.png'
legend_act_png = FloatImage(legend_png, bottom=2, left=2).add_to(main_map)

In [6]:
# Read Excel Sheets

supplier_loc = pd.read_excel("/Users/niallxd/Jupyter Notebooks/Supplier Map/Excel Sheets/Supplier Locations.xlsx")

howarth_branches = pd.read_excel("/Users/niallxd/Jupyter Notebooks/Supplier Map/Excel Sheets/Howarth Branches.xlsx")

supplier_loc

,Supplier Assigned,Postcode,Latitude,Longitude,Margin
0,JONES SKIP HIRE (WOLVERHAMPTON) LIMITED,WV1 2HS,52.582389,-2.107678,20.780099
1,H BROWN AND SON (RECYCLING) LIMITED,ST6 2BY,53.045585,-2.183905,13.130000
2,SMITHS (GLOUCESTER) LTD,GL10 3AQ,51.747963,-2.331469,19.104444
3,ALCHEM (MERSEYSIDE) LTD,CH65 4LB,53.278955,-2.872519,23.628750
4,THOMAS BROS EXCAVATION LTD (LUTON),SG17 5PL,52.035330,-0.384746,25.699286
...,...,...,...,...,...
233,S & B WASTE MANAGEMENT & RECYCLING LTD,B66 2DZ,52.503555,-1.962051,14.150000
234,MERCIAN SKIP HIRE MIDLANDS LIMITED,B30 3JJ,52.416528,-1.919386,26.200000
235,CLEARABEE LTD,B3 1DW,52.484383,-1.903660,0.000000
236,KIELY BROS LIMITED,B25 8HH,52.460443,-1.833244,20.166000


In [7]:
Geocoder(supplier_loc, "Suppliers")
supplier_loc.to_excel("/Users/niallxd/Jupyter Notebooks/Supplier Map/Excel Sheets/Supplier Locations.xlsx", index=False)

Geocoder(howarth_branches, "Howarth Branches")
howarth_branches.to_excel("/Users/niallxd/Jupyter Notebooks/Supplier Map/Excel Sheets/Howarth Branches.xlsx", index=False)

supplier_loc

Beginning Geocoding of Suppliers...
25% Complete
50% Complete
75% Complete
Geocoding Suppliers Complete...
Beginning Geocoding of Howarth Branches...
25% Complete
50% Complete
75% Complete
Geocoding Howarth Branches Complete...


,Supplier Assigned,Postcode,Latitude,Longitude,Margin
0,JONES SKIP HIRE (WOLVERHAMPTON) LIMITED,WV1 2HS,52.582389,-2.107678,20.780099
1,H BROWN AND SON (RECYCLING) LIMITED,ST6 2BY,53.045585,-2.183905,13.130000
2,SMITHS (GLOUCESTER) LTD,GL10 3AQ,51.747963,-2.331469,19.104444
3,ALCHEM (MERSEYSIDE) LTD,CH65 4LB,53.278955,-2.872519,23.628750
4,THOMAS BROS EXCAVATION LTD (LUTON),SG17 5PL,52.035330,-0.384746,25.699286
...,...,...,...,...,...
233,S & B WASTE MANAGEMENT & RECYCLING LTD,B66 2DZ,52.503555,-1.962051,14.150000
234,MERCIAN SKIP HIRE MIDLANDS LIMITED,B30 3JJ,52.416528,-1.919386,26.200000
235,CLEARABEE LTD,B3 1DW,52.484383,-1.903660,0.000000
236,KIELY BROS LIMITED,B25 8HH,52.460443,-1.833244,20.166000


In [8]:
# Add Suppliers To Map

for i in range(0, len(supplier_loc)):
    
    if supplier_loc.at[i, 'Margin'] == 0:
        icon_supplier=folium.Icon(color='darkblue', icon='trash')
    elif supplier_loc.at[i, 'Margin'] >= 15:
        icon_supplier=folium.Icon(color='green', icon='trash')
    elif supplier_loc.at[i, 'Margin'] <= 10:
        icon_supplier=folium.Icon(color='red', icon='trash')
    else:
        icon_supplier=folium.Icon(color='orange', icon='trash') 
    
    
    
    order = folium.Marker(
          location=[supplier_loc.at[i, 'Latitude'], supplier_loc.at[i, 'Longitude']],
          radius=1,
          tooltip=f"""Supplier Name: {supplier_loc.at[i, 'Supplier Assigned']} |
                  Average Margin To Date: {round(supplier_loc.at[i, 'Margin'], 2)}%""",
          popup = folium.Popup(f"""Supplier Name: {supplier_loc.at[i, 'Supplier Assigned']} |
                  Average Margin To Date: {round(supplier_loc.at[i, 'Margin'], 2)}%""", max_width=800),
          icon=icon_supplier
          ).add_to(supplierlayer)


# Add Howarth Branches To Map


for i in range(0, len(howarth_branches)):
    order = folium.Marker(
          location=[howarth_branches.at[i, 'Latitude'], howarth_branches.at[i, 'Longitude']],
          radius=1,
          tooltip=f"""Branch Name: {howarth_branches.at[i, 'Branch Name']}""",
          popup = folium.Popup(f"Branch Name: {howarth_branches.at[i, 'Branch Name']}", max_width=800),
          icon=folium.Icon(color='darkgreen', icon='home')
          ).add_to(howarthlayer)
    
# Add 15 mile rad around Howarth Branches To Map


for i in range(0, len(howarth_branches)):
    order = folium.Circle(
          location=[howarth_branches.at[i, 'Latitude'], howarth_branches.at[i, 'Longitude']],
          radius=15 * 1609.34,
          tooltip=f"""Branch Name: {howarth_branches.at[i, 'Branch Name']}""",
          popup = folium.Popup(f"Branch Name: {howarth_branches.at[i, 'Branch Name']}", max_width=800),
          color='darkgreen', fill_color='green'
          ).add_to(howarthradlayer)


In [9]:
main_map

In [10]:
print("Starting HTML export...")
strt_time = time.time()
main_map.save('index.html')
print("HTML export complete.")
elspased_time = round(time.time() - strt_time, 0)
print(f"Time Taken: {elspased_time}s")

Starting HTML export...
HTML export complete.
Time Taken: 0.0s
